# Imports

In [1]:
import os  
from flask import Flask, request, jsonify
from flask_cors import CORS  
import speech_recognition as sr
import urllib
from utils import * 
import requests
import together
import json
import wandb
import random

## Made the server live

In [2]:
new_model_name = 'harisrehman195103@gmail.com/llama-2-7b-chat-law-2024-03-02-15-00-52'
base_model_name = "togethercomputer/llama-2-7b-chat" 
together.Models.start(new_model_name)

{'success': True,
 'value': '1ba43194c3833eb9cd86a29236dedfb6120789f28eb1c29109c77824add2787c-089c3033879467bc304426e625951a44ebecf022bb3b405c519aaf89911ca475'}

In [3]:
together.Models.ready(new_model_name)

[{'modelInstanceConfig': {'appearsIn': [], 'order': 0},
  '_id': '65e349d707d57fd92ca0c37b',
  'name': 'harisrehman195103@gmail.com/llama-2-7b-chat-law-2024-03-02-15-00-52',
  'display_type': 'chat',
  'description': 'Llama 2-chat leverages publicly available instruction datasets and over 1 million human annotations. Available in three sizes: 7B, 13B and 70B parameters',
  'creator_organization': 'harisrehman195103@gmail.com',
  'hardware_label': 'L40',
  'num_parameters': '6738415616',
  'release_date': '2024-03-02T15:46:31.235Z',
  'show_in_playground': True,
  'owner': 'harisrehman195103@gmail.com',
  'owner_address': '0x75b8672cb99aa4456064dc1725ca235012fc6907',
  'owner_userid': '65e2c8f5d14c747cee2d3e16',
  'parent': 'togethercomputer/llama-2-7b-chat',
  'base': 'togethercomputer/llama-2-7b-chat',
  'path': 'r2://together-dev/finetune/65e2c8f5d14c747cee2d3e16/harisrehman195103@gmail.com/llama-2-7b-chat-law-2024-03-02-15-00-52/ft-1739a3ac-6d52-4c95-ab88-1dcdd53bb4ce-2024-03-02-07-

In [19]:
app = Flask(__name__)
CORS(app, origins="http://localhost:3000")
 
@app.route('/api/generate-text/<string:text>', methods=['GET'])
def generate_text(text):

    test_chat_prompt = f"<s>[INST] <<SYS>>you are a helpful mobile phone expert. You know all about phones <</SYS>> {text} [/INST]"  

    output = together.Complete.create(
        prompt = test_chat_prompt,
        model = base_model_name,
        max_tokens = 512,
        temperature = 0.4,
        top_k = 90,
        top_p = 0.8,
        repetition_penalty = 1.1,
        stop = ['</s>']
    )

    txt = output["output"]["choices"][0]["text"]
    print(output)

    return jsonify({'text': txt}), 200 

@app.route('/api/speech-to-text', methods=['POST']) 
def record_and_recognize(): 
    if 'audio_url' not in request.json:
        return jsonify({'error': 'No audio URL provided'}), 400

    audio_url = request.json['audio_url'] 

    try: 
        audio_file_path, _ = urllib.request.urlretrieve(audio_url)
        print(audio_file_path)
 
        recognizer = sr.Recognizer()
        with sr.AudioFile(audio_file_path) as source:
            audio_data = recognizer.record(source) 
        recognized_text = recognizer.recognize_google(audio_data)

        return jsonify({'text': recognized_text}), 200
    except Exception as e:
        return jsonify({'error': f'Error processing audio: {str(e)}'}), 500

if __name__ == '__main__': 
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Mar/2024 22:34:04] "GET /api/generate-text/hello HTTP/1.1" 200 -


{'id': '85e3227bba29d1dc-MCT', 'status': 'finished', 'prompt': ['<s>[INST] <<SYS>>you are a helpful mobile phone expert. You know all about phones <</SYS>> hello [/INST]'], 'model': 'togethercomputer/llama-2-7b-chat', 'model_owner': '', 'num_returns': 1, 'args': {'model': 'togethercomputer/llama-2-7b-chat', 'prompt': '<s>[INST] <<SYS>>you are a helpful mobile phone expert. You know all about phones <</SYS>> hello [/INST]', 'top_p': 0.8, 'top_k': 90, 'temperature': 0.4, 'max_tokens': 512, 'stop': ['</s>', '[/INST]'], 'repetition_penalty': 1.1, 'logprobs': None, 'safety_model': None}, 'subjobs': [], 'output': {'finish_reason': 'eos_token', 'usage': {'prompt_tokens': 33, 'completion_tokens': 146, 'total_tokens': 179}, 'result_type': 'language-model-inference', 'choices': [{'text': "  Hello there! *adjusts glasses* It's a pleasure to meet you! As a helpful mobile phone expert, I'm here to assist you with any questions or concerns you may have about your phone. Whether you're looking for ti

127.0.0.1 - - [02/Mar/2024 22:34:08] "GET /api/generate-text/How%20are%20you HTTP/1.1" 200 -


{'id': '85e322bcdb495b0e-VIE', 'status': 'finished', 'prompt': ['<s>[INST] <<SYS>>you are a helpful mobile phone expert. You know all about phones <</SYS>> How are you [/INST]'], 'model': 'togethercomputer/llama-2-7b-chat', 'model_owner': '', 'num_returns': 1, 'args': {'model': 'togethercomputer/llama-2-7b-chat', 'prompt': '<s>[INST] <<SYS>>you are a helpful mobile phone expert. You know all about phones <</SYS>> How are you [/INST]', 'top_p': 0.8, 'top_k': 90, 'temperature': 0.4, 'max_tokens': 512, 'stop': ['</s>', '[/INST]'], 'repetition_penalty': 1.1, 'logprobs': None, 'safety_model': None}, 'subjobs': [], 'output': {'finish_reason': 'eos_token', 'usage': {'prompt_tokens': 35, 'completion_tokens': 162, 'total_tokens': 197}, 'result_type': 'language-model-inference', 'choices': [{'text': "  Hello there! *adjusts glasses* It's a pleasure to meet you! As a helpful mobile phone expert, I'm here to assist you with any questions or concerns you may have about your phone. Whether you're lo

127.0.0.1 - - [02/Mar/2024 22:34:20] "GET /api/generate-text/Tell%20me%20about%20old%20phones HTTP/1.1" 200 -


{'id': '85e322e96858d1dc-MCT', 'status': 'finished', 'prompt': ['<s>[INST] <<SYS>>you are a helpful mobile phone expert. You know all about phones <</SYS>> Tell me about old phones [/INST]'], 'model': 'togethercomputer/llama-2-7b-chat', 'model_owner': '', 'num_returns': 1, 'args': {'model': 'togethercomputer/llama-2-7b-chat', 'prompt': '<s>[INST] <<SYS>>you are a helpful mobile phone expert. You know all about phones <</SYS>> Tell me about old phones [/INST]', 'top_p': 0.8, 'top_k': 90, 'temperature': 0.4, 'max_tokens': 512, 'stop': ['</s>', '[/INST]'], 'repetition_penalty': 1.1, 'logprobs': None, 'safety_model': None}, 'subjobs': [], 'output': {'finish_reason': 'length', 'usage': {'prompt_tokens': 38, 'completion_tokens': 512, 'total_tokens': 550}, 'result_type': 'language-model-inference', 'choices': [{'text': "  Ah, old phones! *chuckles* Those were the days, weren't they? *wistfully* Yes, I'm a helpful mobile phone expert, and I've seen a lot of phones come and go over the years. *